In [1]:
import torch
import numpy as np
import tqdm
from wilds import get_dataset
# from wilds.common.data_loaders import get_train_loader
from wilds.common.data_loaders import get_train_loader, get_eval_loader
import transforms
# from transforms import 

from types import SimpleNamespace
from wilds_configs.utils import populate_defaults
from wilds_configs import datasets as dataset_configs
from wilds_configs import model as model_configs

from wilds.datasets.wilds_dataset import WILDSSubset

from wilds_models.initializer import initialize_model
from wilds_algorithms.initializer import infer_d_out
# import configs
# from configs.datasets import dataset_defaults

2022-09-27 00:48:56.401551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-27 00:48:56.530890: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-27 00:48:57.076104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.0/lib64
2022-09-27 00:48:57.076147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: 

In [2]:
config = SimpleNamespace(
    algorithm='ERM',
    load_featurizer_only=False,
    pretrained_model_path=None,
    **dataset_configs.dataset_defaults["civilcomments"],
    )
config.model_kwargs = {}
# config = populate_defaults(config)

In [4]:
dataset = get_dataset(dataset="civilcomments", download=False,
                    #   root_dir='/data/users/pavel_i/datasets/')
                      root_dir='/home/pavel/datasets/')

transform = transforms.initialize_transform(
        transform_name='bert',
        config=config,
        dataset=dataset,
        additional_transform_name=None,
        is_training=True)

train_data = dataset.get_subset(
        "train",
        frac=1.,
        transform=transform)

# Get the test set
test_data = dataset.get_subset(
    "test", transform=transform
)
original_val_data = dataset.get_subset(
    "val", transform=transform
)

In [5]:
seed = 0
idx = train_data.indices.copy()
rng = np.random.default_rng(0)
rng.shuffle(idx)
n_train = int((1 - 0.2) * len(idx))
train_idx = idx[:n_train]
val_idx = idx[n_train:]

val_data = WILDSSubset(
    dataset,
    indices=val_idx,
    transform=transform)

In [8]:
train_data

In [7]:
model = initialize_model(config=config, d_out=infer_d_out(train_data, config))
ckpt_dict = torch.load('ckpts/civilcomments_seed_0_epoch_last_model.pth')
# ckpt_dict = torch.load('logs/civilcomments_dfrdrop_0/civilcomments_seed:0_epoch:last_model.pth')
model.load_state_dict({k[len('model.'):]: v for (k, v) in ckpt_dict['algorithm'].items()})

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN t

<All keys matched successfully>

In [7]:
model.cuda();
model.eval();

In [8]:
train_loader = get_train_loader(
        loader="standard",
        dataset=train_data,
        batch_size=16,
        uniform_over_groups=False,
        grouper=None,
        distinct_groups=False,
        n_groups_per_batch=None)

# Prepare the evaluation data loader
test_loader = get_eval_loader("standard", test_data, batch_size=16)

## Eval

In [14]:
all_y_pred, all_y_true, all_metadata = [], [], []
with torch.no_grad():
    for x, y_true, metadata in tqdm.tqdm(test_loader):
        y_pred = model(x.cuda())
        all_y_pred.append(y_pred.cpu())
        all_y_true.append(y_true.cpu())
        all_metadata.append(metadata)
        # break
all_y_pred = torch.cat(all_y_pred, axis=0)
all_y_true = torch.cat(all_y_true, axis=0)
all_metadata = torch.cat(all_metadata, axis=0)

100%|██████████| 8362/8362 [07:38<00:00, 18.25it/s]


In [15]:
print(dataset.eval(torch.argmax(all_y_pred, axis=1), all_y_true, all_metadata)[-1])

Average acc: 0.920
  male                   acc on non_toxic: 0.938 (n =  12092)    acc on toxic: 0.615 (n =   2203) 
  female                 acc on non_toxic: 0.944 (n =  14179)    acc on toxic: 0.606 (n =   2270) 
  LGBTQ                  acc on non_toxic: 0.874 (n =   3210)    acc on toxic: 0.581 (n =   1216) 
  christian              acc on non_toxic: 0.959 (n =  12101)    acc on toxic: 0.561 (n =   1260) 
  muslim                 acc on non_toxic: 0.899 (n =   5355)    acc on toxic: 0.565 (n =   1627) 
  other_religions        acc on non_toxic: 0.933 (n =   2980)    acc on toxic: 0.567 (n =    520) 
  black                  acc on non_toxic: 0.846 (n =   3335)    acc on toxic: 0.645 (n =   1537) 
  white                  acc on non_toxic: 0.855 (n =   5723)    acc on toxic: 0.643 (n =   2246) 
Worst-group acc: 0.561



## DFR-Val

In [9]:
def get_embeddings(model, loader):
    all_embeddings, all_y_true, all_metadata = [], [], []
    with torch.no_grad():
        for x, y_true, metadata in tqdm.tqdm(loader):
            embeddings = model(x.cuda())
            all_embeddings.append(embeddings.cpu())
            all_y_true.append(y_true.cpu())
            all_metadata.append(metadata)
            # break
    all_embeddings = torch.cat(all_embeddings, axis=0)
    all_y_true = torch.cat(all_y_true, axis=0)
    all_metadata = torch.cat(all_metadata, axis=0)
    return all_embeddings, all_y_true, all_metadata

In [10]:
import sys
sys.path.append('../')
from dfr import dfr_tune, dfr_run, dfr_tune_and_run, dfr_predict

%load_ext autoreload
%autoreload 2

In [11]:
val_loader = get_eval_loader("standard", val_data, batch_size=16)
original_val_loader = get_eval_loader("standard", original_val_data, batch_size=16)
test_loader = get_eval_loader("standard", test_data, batch_size=16)

In [12]:
model.classifier = torch.nn.Identity(model.classifier.in_features)
model.eval();

In [14]:
val_embeddings, val_y_true, val_metadata = get_embeddings(model, val_loader)
original_val_embeddings, original_val_y_true, original_val_metadata = get_embeddings(model, original_val_loader)
test_embeddings, test_y_true, test_metadata = get_embeddings(model, test_loader)

100%|██████████| 8362/8362 [07:38<00:00, 18.24it/s]


### Black-White

In [87]:
val_data.metadata_fields[6]

'black'

In [89]:
val_spurious = val_metadata[:, 6]
val_groups = val_y_true * 2 + val_spurious
print(torch.bincount(val_groups))

tensor([46307,  1349,  5508,   644])


In [90]:
logreg, scaler = dfr_tune_and_run(val_embeddings, val_y_true, val_groups, verbose=True)

1.0: [0.82393819 0.66034985 0.77512839 0.67901235]
0.7: [0.83149171 0.62973761 0.80190756 0.67592593]
0.3: [0.86390711 0.67346939 0.83015407 0.70061728]
0.1: [0.8660221  0.66180758 0.83345561 0.77777778]
0.07: [0.8917904  0.70553936 0.81731475 0.80246914]
0.03: [0.87007942 0.696793   0.83895818 0.7962963 ]
0.01: [0.87357562 0.74927114 0.82978723 0.79012346]
Training model 0/10, group counts: [644 644 644 644]
Training model 1/10, group counts: [644 644 644 644]
Training model 2/10, group counts: [644 644 644 644]
Training model 3/10, group counts: [644 644 644 644]
Training model 4/10, group counts: [644 644 644 644]
Training model 5/10, group counts: [644 644 644 644]
Training model 6/10, group counts: [644 644 644 644]
Training model 7/10, group counts: [644 644 644 644]
Training model 8/10, group counts: [644 644 644 644]
Training model 9/10, group counts: [644 644 644 644]


In [91]:
val_preds = torch.from_numpy(dfr_predict(val_embeddings, logreg, scaler))
test_preds = torch.from_numpy(dfr_predict(test_embeddings, logreg, scaler))

In [92]:
print(dataset.eval(test_preds, test_y_true, test_metadata)[-1])

Average acc: 0.882
  male                   acc on non_toxic: 0.837 (n =  12092)    acc on toxic: 0.798 (n =   2203) 
  female                 acc on non_toxic: 0.845 (n =  14179)    acc on toxic: 0.806 (n =   2270) 
  LGBTQ                  acc on non_toxic: 0.677 (n =   3210)    acc on toxic: 0.799 (n =   1216) 
  christian              acc on non_toxic: 0.890 (n =  12101)    acc on toxic: 0.781 (n =   1260) 
  muslim                 acc on non_toxic: 0.742 (n =   5355)    acc on toxic: 0.784 (n =   1627) 
  other_religions        acc on non_toxic: 0.831 (n =   2980)    acc on toxic: 0.773 (n =    520) 
  black                  acc on non_toxic: 0.737 (n =   3335)    acc on toxic: 0.768 (n =   1537) 
  white                  acc on non_toxic: 0.723 (n =   5723)    acc on toxic: 0.793 (n =   2246) 
Worst-group acc: 0.677



In [93]:
[(test_preds == test_y_true)[test_groups == g].float().mean() for g in range(4)]

[tensor(0.9270), tensor(0.8303), tensor(0.8288), tensor(0.7917)]

### All identity

In [15]:
val_spurious = val_metadata[:, :8]
val_spurious = torch.any(val_spurious, dim=1).int()
val_groups = val_y_true * 2 + val_spurious

test_spurious = test_metadata[:, :8]
test_spurious = torch.any(test_spurious, dim=1).int()
test_groups = test_y_true * 2 + test_spurious

In [23]:
logreg, scaler = dfr_tune_and_run(val_embeddings, val_y_true, val_groups, verbose=True)

1.0: [0.8883061  0.7869163  0.81951975 0.79423403]
0.7: [0.89395468 0.7844645  0.81719597 0.77953646]
0.3: [0.89751866 0.78925666 0.85127808 0.81797626]
0.1: [0.89745142 0.77922657 0.86676995 0.85245902]
0.07: [0.89482886 0.77621754 0.8721921  0.85867722]
0.03: [0.9002757  0.76774769 0.86986832 0.86037309]
0.01: [0.89489611 0.76707901 0.88148722 0.86319955]
Training model 0/10, group counts: [2576 2576 2576 2576]
Training model 1/10, group counts: [2576 2576 2576 2576]
Training model 2/10, group counts: [2576 2576 2576 2576]
Training model 3/10, group counts: [2576 2576 2576 2576]
Training model 4/10, group counts: [2576 2576 2576 2576]
Training model 5/10, group counts: [2576 2576 2576 2576]
Training model 6/10, group counts: [2576 2576 2576 2576]
Training model 7/10, group counts: [2576 2576 2576 2576]
Training model 8/10, group counts: [2576 2576 2576 2576]
Training model 9/10, group counts: [2576 2576 2576 2576]


In [94]:
val_preds = torch.from_numpy(dfr_predict(val_embeddings, logreg, scaler))
original_val_preds = torch.from_numpy(dfr_predict(original_val_embeddings, custom_logreg, scaler))
test_preds = torch.from_numpy(dfr_predict(test_embeddings, logreg, scaler))

In [27]:
print(dataset.eval(test_preds, test_y_true, test_metadata)[-1])

Average acc: 0.855
  male                   acc on non_toxic: 0.805 (n =  12092)    acc on toxic: 0.842 (n =   2203) 
  female                 acc on non_toxic: 0.837 (n =  14179)    acc on toxic: 0.811 (n =   2270) 
  LGBTQ                  acc on non_toxic: 0.600 (n =   3210)    acc on toxic: 0.870 (n =   1216) 
  christian              acc on non_toxic: 0.875 (n =  12101)    acc on toxic: 0.800 (n =   1260) 
  muslim                 acc on non_toxic: 0.688 (n =   5355)    acc on toxic: 0.829 (n =   1627) 
  other_religions        acc on non_toxic: 0.787 (n =   2980)    acc on toxic: 0.827 (n =    520) 
  black                  acc on non_toxic: 0.542 (n =   3335)    acc on toxic: 0.886 (n =   1537) 
  white                  acc on non_toxic: 0.594 (n =   5723)    acc on toxic: 0.876 (n =   2246) 
Worst-group acc: 0.542



In [26]:
[(test_preds == test_y_true)[test_groups == g].float().mean() for g in range(4)]

[tensor(0.8949), tensor(0.7857), tensor(0.8841), tensor(0.8468)]

In [115]:
custom_logreg = copy.deepcopy(logreg)
custom_logreg.intercept_ -= 0.01

In [116]:
val_preds = torch.from_numpy(dfr_predict(val_embeddings, custom_logreg, scaler))
original_val_preds = torch.from_numpy(dfr_predict(original_val_embeddings, custom_logreg, scaler))
test_preds = torch.from_numpy(dfr_predict(test_embeddings, custom_logreg, scaler))

In [117]:
print(dataset.eval(original_val_preds, original_val_y_true, original_val_metadata)[-1])

Average acc: 0.886
  male                   acc on non_toxic: 0.847 (n =   4050)    acc on toxic: 0.832 (n =    715) 
  female                 acc on non_toxic: 0.848 (n =   5120)    acc on toxic: 0.825 (n =    771) 
  LGBTQ                  acc on non_toxic: 0.698 (n =   1099)    acc on toxic: 0.782 (n =    358) 
  christian              acc on non_toxic: 0.912 (n =   4166)    acc on toxic: 0.721 (n =    384) 
  muslim                 acc on non_toxic: 0.743 (n =   1598)    acc on toxic: 0.781 (n =    512) 
  other_religions        acc on non_toxic: 0.829 (n =    824)    acc on toxic: 0.691 (n =    162) 
  black                  acc on non_toxic: 0.724 (n =   1119)    acc on toxic: 0.790 (n =    533) 
  white                  acc on non_toxic: 0.711 (n =   2015)    acc on toxic: 0.783 (n =    852) 
Worst-group acc: 0.691



In [118]:
print(dataset.eval(test_preds, test_y_true, test_metadata)[-1])

Average acc: 0.883
  male                   acc on non_toxic: 0.838 (n =  12092)    acc on toxic: 0.797 (n =   2203) 
  female                 acc on non_toxic: 0.847 (n =  14179)    acc on toxic: 0.804 (n =   2270) 
  LGBTQ                  acc on non_toxic: 0.680 (n =   3210)    acc on toxic: 0.798 (n =   1216) 
  christian              acc on non_toxic: 0.891 (n =  12101)    acc on toxic: 0.776 (n =   1260) 
  muslim                 acc on non_toxic: 0.747 (n =   5355)    acc on toxic: 0.781 (n =   1627) 
  other_religions        acc on non_toxic: 0.833 (n =   2980)    acc on toxic: 0.771 (n =    520) 
  black                  acc on non_toxic: 0.739 (n =   3335)    acc on toxic: 0.764 (n =   1537) 
  white                  acc on non_toxic: 0.725 (n =   5723)    acc on toxic: 0.790 (n =   2246) 
Worst-group acc: 0.680



In [ ]:
custom_logreg = copy.deepcopy(logreg)
custom_logreg.intercept_ -= 0.1

val_preds = torch.from_numpy(dfr_predict(val_embeddings, custom_logreg, scaler))
original_val_preds = torch.from_numpy(dfr_predict(original_val_embeddings, custom_logreg, scaler))
test_preds = torch.from_numpy(dfr_predict(test_embeddings, custom_logreg, scaler))

print(dataset.eval(original_val_preds, original_val_y_true, original_val_metadata)[-1])

print(dataset.eval(test_preds, test_y_true, test_metadata)[-1])

### Per-identity

In [16]:
identity_counts = val_metadata[:, :8].sum(axis=0)

In [17]:
identity_ordering = torch.argsort(identity_counts)
identity_counts[identity_ordering]

tensor([1293, 1677, 1993, 2755, 3395, 5182, 5980, 7199])

In [18]:
[dataset.metadata_fields[i] for i in identity_ordering]

['other_religions',
 'LGBTQ',
 'black',
 'muslim',
 'white',
 'christian',
 'male',
 'female']

In [19]:
val_metadata_ordered = val_metadata[:, :8].T[identity_ordering].T

In [20]:
groups = torch.argmax(torch.cat(
        [torch.zeros((len(val_metadata_ordered),))[:, None], val_metadata_ordered], axis=1), axis=1)
groups = groups * 2 + val_y_true
torch.bincount(groups)

tensor([29762,  2576,  1092,   201,  1159,   466,  1257,   584,  1843,   460,
         1805,   624,  3750,   258,  3703,   505,  3285,   478])

In [21]:
logreg, scaler = dfr_tune_and_run(val_embeddings, val_y_true, groups, verbose=True)

1.0: [0.88929293 0.80061586 0.79118774 0.75531915 0.70819113 0.71487603
 0.71186441 0.72697368 0.70171674 0.73305085 0.68574635 0.73968254
 0.85957447 0.70542636 0.82158351 0.77777778 0.83695652 0.71551724]
0.7: [0.89279461 0.79368745 0.77203065 0.76595745 0.71672355 0.70247934
 0.72265023 0.69078947 0.70493562 0.75847458 0.72615039 0.78412698
 0.87819149 0.75193798 0.84598698 0.75720165 0.83031401 0.80172414]
0.3: [0.91023569 0.82525019 0.82758621 0.77659574 0.69112628 0.75619835
 0.74730354 0.73355263 0.70708155 0.81355932 0.73737374 0.78412698
 0.90904255 0.75193798 0.87472885 0.76954733 0.84722222 0.80172414]
0.1: [0.90686869 0.83294842 0.8256705  0.75531915 0.72696246 0.75619835
 0.74422188 0.6875     0.73712446 0.80084746 0.75869809 0.78730159
 0.8962766  0.78294574 0.835141   0.79835391 0.82789855 0.8362069 ]
0.07: [0.90296296 0.86297152 0.82183908 0.76595745 0.70477816 0.77272727
 0.72727273 0.73355263 0.73175966 0.76271186 0.75196409 0.80634921
 0.89574468 0.7751938  0.8432754

In [33]:
import copy

In [81]:
custom_logreg = copy.deepcopy(logreg)
custom_logreg.intercept_ -= 0.1

In [82]:
val_preds = torch.from_numpy(dfr_predict(val_embeddings, custom_logreg, scaler))
original_val_preds = torch.from_numpy(dfr_predict(original_val_embeddings, custom_logreg, scaler))
test_preds = torch.from_numpy(dfr_predict(test_embeddings, custom_logreg, scaler))

In [83]:
print(dataset.eval(original_val_preds, original_val_y_true, original_val_metadata)[-1])

Average acc: 0.880
  male                   acc on non_toxic: 0.850 (n =   4050)    acc on toxic: 0.815 (n =    715) 
  female                 acc on non_toxic: 0.853 (n =   5120)    acc on toxic: 0.812 (n =    771) 
  LGBTQ                  acc on non_toxic: 0.758 (n =   1099)    acc on toxic: 0.721 (n =    358) 
  christian              acc on non_toxic: 0.910 (n =   4166)    acc on toxic: 0.750 (n =    384) 
  muslim                 acc on non_toxic: 0.743 (n =   1598)    acc on toxic: 0.771 (n =    512) 
  other_religions        acc on non_toxic: 0.825 (n =    824)    acc on toxic: 0.716 (n =    162) 
  black                  acc on non_toxic: 0.727 (n =   1119)    acc on toxic: 0.779 (n =    533) 
  white                  acc on non_toxic: 0.730 (n =   2015)    acc on toxic: 0.764 (n =    852) 
Worst-group acc: 0.716



In [84]:
print(dataset.eval(test_preds, test_y_true, test_metadata)[-1])

Average acc: 0.876
  male                   acc on non_toxic: 0.841 (n =  12092)    acc on toxic: 0.791 (n =   2203) 
  female                 acc on non_toxic: 0.851 (n =  14179)    acc on toxic: 0.792 (n =   2270) 
  LGBTQ                  acc on non_toxic: 0.745 (n =   3210)    acc on toxic: 0.739 (n =   1216) 
  christian              acc on non_toxic: 0.882 (n =  12101)    acc on toxic: 0.783 (n =   1260) 
  muslim                 acc on non_toxic: 0.756 (n =   5355)    acc on toxic: 0.765 (n =   1627) 
  other_religions        acc on non_toxic: 0.829 (n =   2980)    acc on toxic: 0.773 (n =    520) 
  black                  acc on non_toxic: 0.740 (n =   3335)    acc on toxic: 0.753 (n =   1537) 
  white                  acc on non_toxic: 0.736 (n =   5723)    acc on toxic: 0.776 (n =   2246) 
Worst-group acc: 0.736



In [37]:
val_preds = torch.from_numpy(dfr_predict(val_embeddings, logreg, scaler))
original_val_preds = torch.from_numpy(dfr_predict(original_val_embeddings, logreg, scaler))
test_preds = torch.from_numpy(dfr_predict(test_embeddings, logreg, scaler))

In [38]:
print(dataset.eval(test_preds, test_y_true, test_metadata)[-1])

Average acc: 0.868
  male                   acc on non_toxic: 0.827 (n =  12092)    acc on toxic: 0.811 (n =   2203) 
  female                 acc on non_toxic: 0.837 (n =  14179)    acc on toxic: 0.811 (n =   2270) 
  LGBTQ                  acc on non_toxic: 0.713 (n =   3210)    acc on toxic: 0.768 (n =   1216) 
  christian              acc on non_toxic: 0.869 (n =  12101)    acc on toxic: 0.809 (n =   1260) 
  muslim                 acc on non_toxic: 0.727 (n =   5355)    acc on toxic: 0.789 (n =   1627) 
  other_religions        acc on non_toxic: 0.809 (n =   2980)    acc on toxic: 0.802 (n =    520) 
  black                  acc on non_toxic: 0.715 (n =   3335)    acc on toxic: 0.785 (n =   1537) 
  white                  acc on non_toxic: 0.709 (n =   5723)    acc on toxic: 0.803 (n =   2246) 
Worst-group acc: 0.709



In [39]:
print(dataset.eval(original_val_preds, original_val_y_true, original_val_metadata)[-1])

Average acc: 0.873
  male                   acc on non_toxic: 0.834 (n =   4050)    acc on toxic: 0.845 (n =    715) 
  female                 acc on non_toxic: 0.837 (n =   5120)    acc on toxic: 0.829 (n =    771) 
  LGBTQ                  acc on non_toxic: 0.732 (n =   1099)    acc on toxic: 0.754 (n =    358) 
  christian              acc on non_toxic: 0.901 (n =   4166)    acc on toxic: 0.771 (n =    384) 
  muslim                 acc on non_toxic: 0.722 (n =   1598)    acc on toxic: 0.795 (n =    512) 
  other_religions        acc on non_toxic: 0.806 (n =    824)    acc on toxic: 0.728 (n =    162) 
  black                  acc on non_toxic: 0.695 (n =   1119)    acc on toxic: 0.801 (n =    533) 
  white                  acc on non_toxic: 0.700 (n =   2015)    acc on toxic: 0.795 (n =    852) 
Worst-group acc: 0.695



In [28]:
print(dataset.eval(val_preds, val_y_true, val_metadata)[-1])

Average acc: 0.869
  male                   acc on non_toxic: 0.823 (n =   5076)    acc on toxic: 0.800 (n =    904) 
  female                 acc on non_toxic: 0.832 (n =   6208)    acc on toxic: 0.825 (n =    991) 
  LGBTQ                  acc on non_toxic: 0.738 (n =   1199)    acc on toxic: 0.793 (n =    478) 
  christian              acc on non_toxic: 0.877 (n =   4709)    acc on toxic: 0.774 (n =    473) 
  muslim                 acc on non_toxic: 0.740 (n =   2160)    acc on toxic: 0.818 (n =    595) 
  other_religions        acc on non_toxic: 0.823 (n =   1092)    acc on toxic: 0.786 (n =    201) 
  black                  acc on non_toxic: 0.723 (n =   1349)    acc on toxic: 0.752 (n =    644) 
  white                  acc on non_toxic: 0.724 (n =   2446)    acc on toxic: 0.803 (n =    949) 
Worst-group acc: 0.723



In [25]:
[(val_preds == val_y_true)[groups == g].float().mean() for g in range(8)]

[tensor(0.9076),
 tensor(0.8661),
 tensor(0.8233),
 tensor(0.7861),
 tensor(0.7386),
 tensor(0.7897),
 tensor(0.7319),
 tensor(0.7551)]

## Remove subset

In [181]:
train_data = dataset.get_subset(
        "train",
        frac=1.,
        transform=transform)

In [184]:
train_data.indices.size

269038

In [189]:
train_data.metadata_array[:, :8].float().mean(axis=0)

tensor([0.1108, 0.1347, 0.0313, 0.0994, 0.0519, 0.0243, 0.0368, 0.0621])